## Multi-Label Auto-Tagger

### Automatic Comment tagging

The purpose of this document is to create an automated tagging system that tags a comment as belonging to a small set of predetermined categories.
Once comment can have multiple tags hence the multilabeling approach.


Example: {'Fantastic meals....quite good service':['food','service']}


In [23]:
###Importations
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.utils import shuffle
from sklearn import metrics

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer

import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.pipeline import Pipeline
import seaborn as sns
from xgboost.sklearn import XGBClassifier
import pickle

## Train

#### a.) Load the Data

In [24]:
csvdata = pd.read_csv('datasets/KCB-Themes_Data_V1 .csv')
csvdata.head()

,commid,rating,comment,theme
0,2.547120e+11,10.0,"easy to access through phone, its everywhere ...",Execution_Excellence
1,2.547120e+11,10.0,"easy to access through phone, its everywhere ...",Products_&_Services
2,2.547120e+11,10.0,"easy to access through phone, its everywhere ...",Transparency
3,2.547220e+11,2.0,#NAME?,Customer_Service
4,2.547120e+11,9.0,#NAME?,Products_&_Services


In [25]:
#Clean up the comments by removing everything that is not a letter, a number of punction

z = lambda x: str(re.sub("[^a-zA-Z0-9\s,!.]", "", x)) #Clean up: replace everything that's not a number or a dot with a space
csvdata['clean_comments'] = ([z(i) for i in  csvdata['comment']]) #Create a new column with clean data
csvdata.head()

,commid,rating,comment,theme,clean_comments
0,2.547120e+11,10.0,"easy to access through phone, its everywhere ...",Execution_Excellence,"easy to access through phone, its everywhere ..."
1,2.547120e+11,10.0,"easy to access through phone, its everywhere ...",Products_&_Services,"easy to access through phone, its everywhere ..."
2,2.547120e+11,10.0,"easy to access through phone, its everywhere ...",Transparency,"easy to access through phone, its everywhere ..."
3,2.547220e+11,2.0,#NAME?,Customer_Service,NAME
4,2.547120e+11,9.0,#NAME?,Products_&_Services,NAME


In [26]:
#Add an extra column for themes as a category
#Label encoding to represent each of the theme classes as numbers
theme_categories = csvdata['theme'].astype('category') #1. We first convert the column into a category
csvdata['theme_categories']  = theme_categories.cat.codes #2. assign the encoded variable to a new column using the cat.codes
target_names = list(theme_categories.cat.categories)
csvdata.head()

,commid,rating,comment,theme,clean_comments,theme_categories
0,2.547120e+11,10.0,"easy to access through phone, its everywhere ...",Execution_Excellence,"easy to access through phone, its everywhere ...",1
1,2.547120e+11,10.0,"easy to access through phone, its everywhere ...",Products_&_Services,"easy to access through phone, its everywhere ...",2
2,2.547120e+11,10.0,"easy to access through phone, its everywhere ...",Transparency,"easy to access through phone, its everywhere ...",4
3,2.547220e+11,2.0,#NAME?,Customer_Service,NAME,0
4,2.547120e+11,9.0,#NAME?,Products_&_Services,NAME,2


In [27]:
# df = pd.Series(csvdata)
# df.describe()

csvdata.describe()
csvdata['theme'].value_counts()


Customer_Service           31166
Execution_Excellence       17484
Products_&_Services        15399
Value_for_Money            10503
Transparency                7099
Relationship_Management     2855
Name: theme, dtype: int64

In [28]:
target_names, len(target_names)

(['Customer_Service',
  'Execution_Excellence',
  'Products_&_Services',
  'Relationship_Management',
  'Transparency',
  'Value_for_Money'],
 6)

In [29]:
processed_data = {}

for row in csvdata.iterrows():
    if row[1]['comment']  in processed_data.keys():  
        processed_data[row[1]['comment']].append(row[1]['theme_categories'])
    else:
        processed_data[row[1]['comment']] = [row[1]['theme_categories']]
        


In [30]:
my_data = {}
my_data['data'] = processed_data.keys()
my_data['target'] = processed_data.values()

X = my_data['data']
y = MultiLabelBinarizer().fit_transform(processed_data.values())
y.shape


(50714, 6)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42,shuffle=True )

Try out some classifiers:
1. Logistic Regression
2. Random Forest
3. SVM
4. Multinomial Naives Bayes
5. XGBoost

### Logistic Regression Classifier

In [32]:
from sklearn.linear_model import LogisticRegression

LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
print('... Processing')
LogReg_pipeline.fit(X_train, y_train)
# compute the testing accuracy
prediction = LogReg_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))


... Processing
Test accuracy is 0.919750246467


### Random Forest Classifier

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


In [34]:
RandomForest_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(RandomForestClassifier(max_depth=150, random_state=0), n_jobs=1)),
            ])
print('... Processing')
RandomForest_pipeline.fit(X_train, y_train)
# compute the testing accuracy
prediction = RandomForest_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))

... Processing
Test accuracy is 0.859217877095


### SVM Classifier

In [35]:
from sklearn import svm

SVM_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(svm.SVC(decision_function_shape='ovo'), n_jobs=1)),
            ])
print('... Processing')
SVM_pipeline.fit(X_train, y_train)
# compute the testing accuracy
prediction = SVM_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))


... Processing
Test accuracy is 0.246795925074


### Naives Bayes Classifier

In [36]:
from sklearn.naive_bayes import MultinomialNB
Naives_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=1)),
            ])
print('... Processing')
Naives_pipeline.fit(X_train, y_train)
# compute the testing accuracy
prediction = Naives_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))

... Processing
Test accuracy is 0.608544199803


#### XGBoost Classifier

In [37]:
classes = len(csvdata['theme'].unique())#number of classes
reg_lambda = 2 #XG Boost's L2 regularization term on weights, increasing it makes the model more conservative.default=1


XGB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(
                    XGBClassifier(                  
                        objective = "multi:softmax", 
                        seed =27,
                        reg_lambda=reg_lambda,
                        num_class = classes
                ), n_jobs=1)),
            ])
print('... Processing')
XGB_pipeline.fit(X_train, y_train)
# compute the testing accuracy
prediction = XGB_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))


... Processing
Test accuracy is 0.866053236937


### Validation

In [38]:
#Validation using the best classifier 
comment1 = 'Make me understand, why are there 2 teller on duty at kikuyu kcb bank,av been here for close to 1 hr waiting and the lobby is almost full?' #
comment2 = 'I received a text of qualifying for loan but every time I request there are incoveniences'
comment3 = 'I misplaced my ATM'
comment4 = 'I am one dissapointed customer lately.Every time I transfer money from Mpesa to my account the transaction is not completed,meaning this cash doesn\'t hit my account'
comment5 = 'CB mobile app disappointing at its best and when you least expect it. The error message that keeps popping out is not funny'
comment6 = "Your service is soooooo slow, went to Karen branch no movement for 30 minutes then since was getting late decided to try your ongata Rongai branch, let's just say been here 1 hour but alas....... I hope never to need to visit your branch again in the near future."


comments_new = [comment1,comment2,comment3,comment4, comment5, comment6]


predicted = LogReg_pipeline.predict(comments_new)
predicted = pd.DataFrame(predicted, columns=target_names)

predicted
   

for tw, category in zip(comments_new, predicted.iterrows()):
    themes=[]
    for i in range(len(category[1])) :
        if category[1][i] == 1:
            themes.append(target_names[i])
    print('\n%r ===> %s' % (tw, themes))



'Make me understand, why are there 2 teller on duty at kikuyu kcb bank,av been here for close to 1 hr waiting and the lobby is almost full?' ===> ['Customer_Service', 'Execution_Excellence']

'I received a text of qualifying for loan but every time I request there are incoveniences' ===> ['Execution_Excellence']

'I misplaced my ATM' ===> ['Products_&_Services']

"I am one dissapointed customer lately.Every time I transfer money from Mpesa to my account the transaction is not completed,meaning this cash doesn't hit my account" ===> ['Execution_Excellence', 'Products_&_Services', 'Value_for_Money']

'CB mobile app disappointing at its best and when you least expect it. The error message that keeps popping out is not funny' ===> ['Products_&_Services']

"Your service is soooooo slow, went to Karen branch no movement for 30 minutes then since was getting late decided to try your ongata Rongai branch, let's just say been here 1 hour but alas....... I hope never to need to visit your branc

#### Model Persistence

In [19]:
filename = 'KCB.sav'
pickle.dump(LogReg_pipeline, open(filename, 'wb'))